# TD3 — Choix architecture OLAP (HOLAP recommandé)

## Contexte métier

- Volumétrie : 50M lignes/an, 5 ans historique.
- Besoins : tableaux de bord récents (60j) <5s ; ad hoc <20s.
- Contraintes : budget stockage limité.

In [ ]:
# Bloc SQLite minimal (illustration, non obligatoire pour le TD conceptuel)
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Mini-jeu de données pour comparer latence/costes fictifs
cursor.execute('CREATE TABLE bench_archi (archi TEXT, scope TEXT, latence_s REAL, cout_unite TEXT)')
cursor.executemany(
    'INSERT INTO bench_archi VALUES (?, ?, ?, ?)',
    [
        ('ROLAP', 'historique', 18.5, 'faible'),
        ('MOLAP', 'recent', 3.2, 'élevé'),
        ('HOLAP', 'mixte', 6.5, 'moyen')
    ]
)
conn.commit()

print('Jeu de comparaison chargé (ROLAP/MOLAP/HOLAP).')

## Diagramme de flux (Mermaid → export SVG si besoin)
```mermaid
flowchart TD
  A[Systèmes sources] --> B[Staging]
  B --> C[Entrepôt ROLAP (détails)]
  C --> D[Cubes MOLAP (agrégés récents)]
  D --> E[BI/Tableaux de bord]
  C --> F[Requêtes ad hoc ROLAP]
```
> Conseils : garder la plage MOLAP réduite pour maîtriser le stockage ; historiser en ROLAP pour l’ad hoc.

## Comparatif architectures

In [ ]:
# Tableau comparatif ROLAP/MOLAP/HOLAP
import pandas as pd

data = {
    'Critère': ['Latence', 'Coût stockage', 'Gouvernance'],
    'ROLAP': ['Moyenne (<20s)', 'Faible', 'Flexible'],
    'MOLAP': ['Excellente (<5s)', 'Élevé', 'Pré-agrégés'],
    'HOLAP': ['Excellente récent, moyenne hist', 'Moyen', 'Mix équilibré']
}

df = pd.DataFrame(data)
print(df.to_markdown(index=False))

## Choix HOLAP

- Détails en ROLAP (historique).
- Agrégés en MOLAP (récents).
- Justification : compromis coût/latence.

## Schéma de flux

```mermaid
flowchart TD
  A[Systèmes sources] --> B[Staging]
  B --> C[Entrepôt ROLAP (détails)]
  C --> D[Cubes MOLAP (agrégés récents)]
  D --> E[BI/Tableaux de bord]
  C --> F[Requêtes ad hoc ROLAP]
```

## Plan actualisation et SLA

- Actualisation : daily <2h pour récents ; hebdo pour historique.
- SLA : <5s tableaux ; <20s ad hoc.

## Risques

1. Stockage : limiter plage MOLAP.
2. Cohérence : ETL + rebuild.
3. Performance : monitor SLA.